In [19]:
from typing import TypedDict, List, Tuple
from langchain_openai import AzureChatOpenAI
from langgraph.graph import StateGraph, END
import os


llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_KEY"],
    deployment_name=os.environ["AZURE_OPENAI_DEPLOYMENT"],
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "2023-05-15")
)

In [17]:
# --- 1. Define your conversation state schema
class ChatState(TypedDict):
    history: List[Tuple[str, str]]  # [("user", "message"), ("assistant", "message"), ...]
    input: str                      # latest user input


In [23]:
def chat_step(state: ChatState) -> ChatState:
    messages = [{"role": "system", "content": "You are a helpful assistant."}]
    for speaker, text in state["history"]:
        messages.append({"role": speaker, "content": text})
    messages.append({"role": "user", "content": state["input"]})
    response = llm.invoke(messages)
    reply = response.content
    # Update conversation history
    state["history"].append(("user", state["input"]))
    state["history"].append(("assistant", reply))
    return state

In [24]:
workflow = StateGraph(state_schema=ChatState)
workflow.add_node("chat", chat_step)
workflow.add_edge("chat", END)
workflow.set_entry_point("chat")
app = workflow.compile()   # <-- This is KEY!

In [25]:
state = ChatState(history=[], input="Who won the 2018 FIFA World Cup?")
state = app.invoke(state)
print("AI:", state["history"][-1][1])  # prints the assistant's answer


AI: The **2018 FIFA World Cup** was won by **France**. They defeated **Croatia** 4–2 in the final, which was held on **July 15, 2018**, at the Luzhniki Stadium in **Moscow, Russia**. This victory marked France's second World Cup title, with their first being in **1998**.


In [26]:
state["input"] = "Who was the captain of that team?"
state = app.invoke(state)
print("AI:", state["history"][-1][1])

AI: The captain of the **France** team that won the 2018 FIFA World Cup was **Hugo Lloris**, the team's goalkeeper. Lloris played a crucial role in the tournament, providing leadership and delivering important performances throughout, including key saves in several matches.


In [27]:
state["input"] = "Where was the final played?"
state = app.invoke(state)
print("AI:", state["history"][-1][1])

AI: The final of the **2018 FIFA World Cup** was played at the **Luzhniki Stadium** in **Moscow, Russia** on **July 15, 2018**. This iconic stadium, the largest in Russia with a capacity of approximately 78,000 spectators, served as the centerpiece venue for the tournament.


Memory based questions using lang graph & uuid

In [6]:
import uuid
from IPython.display import Image, display
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_openai import AzureChatOpenAI  # Use Azure model
import os

Start a new graph

In [4]:
# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

Load environment variables

In [7]:
# Define a chat model (Azure version)
model = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_KEY"],
    deployment_name=os.environ["AZURE_OPENAI_DEPLOYMENT"],
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "2023-05-15")
)

Function for chat node

In [8]:
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    # We return a list, because this will get added to the existing list
    return {"messages": response}

Add Start node to graph workflow

In [9]:
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

Creates a memory for conversation

In [10]:
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


Creates a unique thread id for maintaining the isolation of memory

In [11]:
thread_id = uuid.uuid4()
config = {"configurable": {"thread_id": thread_id}}

In [12]:
input_message = HumanMessage(content="hi! I'm bob")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()


================================ Human Message =================================

hi! I'm bob
================================== Ai Message ==================================

Hi, Bob! 😊 How’s it going? What can I help you with today?


In [13]:
input_message = HumanMessage(content="what was my name?")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

what was my name?
================================== Ai Message ==================================

Your name is Bob! 😊 Did I get it right?


In [14]:
input_message = HumanMessage(content="what was my age?")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

what was my age?
================================== Ai Message ==================================

You haven’t mentioned your age yet, Bob! 😊 But feel free to share if you’d like, or let me know how I can help you!
